In [ ]:
import matplotlib,numpy,astropy
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astroquery.vizier import Vizier
from astroquery.ned import Ned
from astropy.table import QTable, join
from astropy import units
from astropy.coordinates import SkyCoord
import pycf3, requests,json,tqdm

In [ ]:
c = 299792.458 * units.km/units.s #km/s
#HI line rest frequency
f0 = 1.420405751 * units.GHz #GHz

In [ ]:
def logMHI_pred(B_c,D,B_c_err,D_err):
    '''
    Predicts HI mass of a galaxy given its B-band luminosity following Jones et al. 2018, A&A 609A, 17J.
    
    Inputs:
        B_c = Extinction and k-corrected B-band apparent magnitude.
        D = Source luminoisty distance [Mpc].
        B_c_err = Error in B_c.
        
    Outputs:
        logMHI = Logarithm of the predicted HI mass [Msol].
        logMHI_err = Error in logMHI.
    '''
    
    logD = numpy.log10(D)
    
    logLB = 10. + 2.*numpy.log10(D) + 0.4*(4.88-B_c)
    
    logMHI = 0.94*logLB + 0.18
    
    logMHI_err = numpy.sqrt(0.2**2. + (0.94**2.)*((0.4*B_c_err)**2.))
    
    return logMHI,logMHI_err

def logMHI_pred_type(B_c,D,T):
    '''
    Predicts HI mass of a galaxy given its B-band luminosity following Jones et al. 2018, A&A 609A, 17J.
    Includes a correction for the galaxy T type, but does not estimate uncertainty.
    
    Inputs:
        B_c = Extinction and k-corrected B-band apparent magnitude.
        D = Source luminoisty distance [Mpc].
        T = Type as in RC3.
        
    Outputs:
        logMHI = Logarithm of the predicted HI mass [Msol].
    '''
    
    logD = numpy.log10(D)
    
    logLB = 10. + 2.*numpy.log10(D) + 0.4*(4.88-B_c)
    logLB = numpy.array(logLB,dtype='float')
    
    logMHI = numpy.zeros(len(logLB))
    
    inx = numpy.where(T < 3)[0]
    logMHI[inx] = 1.46*logLB[inx] - 5.38
    inx = numpy.where(numpy.logical_and(T >= 3, T <= 5))[0]
    logMHI[inx] = 1.59*logLB[inx] - 6.45
    inx = numpy.where(T > 5)[0]
    logMHI[inx] = 0.78*logLB[inx] + 1.95
    
    return logMHI

def logMHI_pred_HG84(B_c,D):
    '''
    Predicts HI mass of a galaxy given its B-band luminosity following Haynes & Giovanelli 1984.
    
    Inputs:
        B_c = Extinction and k-corrected B-band apparent magnitude.
        D = Source luminoisty distance [Mpc].
        
    Outputs:
        logMHI = Logarithm of the predicted HI mass [Msol].
    '''
    
    logD = numpy.log10(D)
    
    logLB = 10. + 2.*numpy.log10(D) + 0.4*(4.88-B_c)
    
    logMHI = 0.66*logLB + 3.17
    
    return logMHI

def DVcalculator(alpha, delta, system='supergalactic', parameter='distance', value=20, calculator='NAM'):
    
    """
    Inputs: 
        alpha: (float) [deg]
            first coordinate parameter  (RA,  Glon, SGL)
        delta: (float) [deg]
            second coordinate parameter (Dec, Glat, SGB)  
        system: (string)
            coordinate system: 
            Options are:
                "equatorial"
                "galactic"
                "supergalactic"
        parameter: (string)
            the quantity whose value is provided
            Options are:
                "distance"
                "velocity"
        value: (float)
            the value of the input quantity
            distance in [Mpc] and velocity in [km/s]
            
        calcualtor: desired Cosmicflows caluclator
            Options are:
                "NAM" to query the calcualtor at http://edd.ifa.hawaii.edu/NAMcalculator
                "CF3" to query the calcualtor at http://edd.ifa.hawaii.edu/CF3calculator
        
    Output:
        A python dictionary which contains the distance and velocity of the 
        given object and the coordinate of the object in different systems

    """
    
    coordinate = [float(alpha), float(delta)]
    query  = {
              'coordinate': coordinate,
              'system': system,
              'parameter': parameter,
              'value': float(value)
             }
    headers = {'Content-type': 'application/json'}
    
    API_url = 'http://edd.ifa.hawaii.edu/'+calculator+'calculator/api.php'
    
    try:
        r = requests.get(API_url, data=json.dumps(query), headers=headers)
        output = json.loads(r.text) # a python dictionary
    except:
        print("Something went wrong!")  
        print("Please check your intput parameters ...")
        output = None

    return output

In [ ]:
VLA_HCGs = [2, 7, 10, 15, 16, 19, 22, 23, 25, 26, 30, 31, 33, 37, 38, 40, 47, 48, 49, 54, 56, 57, 58, 59, 61, 62, 68, 71, 79, 88, 89, 90, 91, 92, 93, 95, 96, 97, 100]

#Use Cosmic Flows 3 API (pycf3-2020.12) to calculate distances
cf3 = pycf3.CF3() 

In [ ]:
HCG_pos = Vizier(catalog='VII/213/groups', columns=['HCG', '_RAJ2000', '_DEJ2000'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]
HCG_pos.add_index('HCG')
HCG_z = Vizier(catalog='VII/213/dynamics', columns=['HCG', 'z'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]
HCG_z.add_index('HCG')

In [ ]:
HCGs = join(HCG_pos,HCG_z,join_type='left')
HCGs.add_index('HCG')

HCGs.rename_column('_RAJ2000', 'ra')
HCGs.rename_column('_DEJ2000', 'dec')

In [ ]:
#Unfortunately HCG 19 has no redshift in Hickson 1992
#Use NED redshift
HCGs.loc[19]['z'] = float(Ned.query_object("HCG19")['Redshift'])

#Manually set the velocity of HCG 48 (wrong in both NED and the original HCG catalogue)
HCGs.loc[48]['z'] = 2352.*(units.km/units.s)/c

In [ ]:
HCGs['cz'] = c*HCGs['z']

In [ ]:
#There is an error in the coordinates of HCG 48
HCGs.loc[48]['ra'] = 159.440201

In [ ]:
#Calculate distances
HCGs['dist'] = numpy.zeros(len(HCGs))
HCGs['dist'].unit = units.Mpc

for HCG in tqdm.tqdm(HCGs['HCG']):
    HCGs.loc[HCG]['dist'] = DVcalculator(HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], system='equatorial', 
             parameter='velocity', value=HCGs.loc[HCG]['cz'], calculator='CF3')['observed']['distance'][0]

In [ ]:
HCG_mems = Vizier(catalog='VII/213/galaxies', columns=['HCG', 'm_HCG', '_RAJ2000', '_DEJ2000', 'MType', 'a', 'b', 'RVhel', 'BTmag', 'e_BTmag', 'BTc'], row_limit=-1).query_constraints(HCG=','.join(numpy.array(VLA_HCGs,dtype='str')))[0]

HCG_mems.rename_column('_RAJ2000', 'ra')
HCG_mems.rename_column('_DEJ2000', 'dec')
HCG_mems.rename_column('RVhel', 'Vel')

In [ ]:
#Remove false group members

gals_to_remove = {2: ['d'], 19: ['d'], 22: ['d', 'e'], 23: ['e'], 25: ['c', 'e', 'g'], 31: ['d'], 38: ['d'], 48: ['c','d'], 59: ['e'], 61: ['b'], 
                  71: ['d'], 79: ['e'], 92: ['a'], 93: ['e']}

In [ ]:
mem_names = []
for i in range(len(HCG_mems['HCG'])):
    mem_names.append('HCG'+str(HCG_mems[i]['HCG'])+str(HCG_mems[i]['m_HCG']))
HCG_mems['name'] = mem_names

In [ ]:
mask = []
for HCG in gals_to_remove.keys():
    for mem in gals_to_remove[HCG]:
        for i in range(len(HCG_mems['HCG'])):
            if int(HCG_mems[i]['HCG']) == HCG and str(HCG_mems[i]['m_HCG']) == mem:
                mask.append(i)
                break
HCG_mems.remove_rows(mask)
HCG_mems.add_index('HCG')

#HCG_mems['m_HCG'] = HCG_mems['m_HCG'].astype('str')

In [ ]:
#Replace bad redshift measurements with NED values

inx1 = numpy.where(HCG_mems['name'] == 'HCG100d')[0][0]
inx2 = 7 #Velocities

HCG_mems[inx1][inx2] = int(numpy.round(c.value*float(Ned.query_object("HCG100d")['Redshift']),decimals=0))

inx1 = numpy.where(HCG_mems['name'] == 'HCG19c')[0][0]

HCG_mems[inx1][inx2] = int(numpy.round(c.value*float(Ned.query_object("HCG19c")['Redshift']),decimals=0))

inx1 = numpy.where(HCG_mems['name'] == 'HCG57h')[0][0]

HCG_mems[inx1][inx2] = int(numpy.round(c.value*float(Ned.query_object("HCG57h")['Redshift']),decimals=0))

inx1 = numpy.where(HCG_mems['name'] == 'HCG48a')[0][0]

HCG_mems[inx1][inx2] = int(numpy.round(c.value*float(Ned.query_object("HCG48a")['Redshift']),decimals=0))

inx1 = numpy.where(HCG_mems['name'] == 'HCG48b')[0][0]

HCG_mems[inx1][inx2] = int(numpy.round(c.value*float(Ned.query_object("HCG48b")['Redshift']),decimals=0))

In [ ]:
HCG_mems

In [ ]:
#Values for added galaxies are from HyperLEDA which unfortunately is not currently searchable with astroquery
HCG_mems.add_row([16,'',32.5734980,-10.3214486,'SBab',None,None,3992,13.66,0.1,13.37,'NGC848'])

HCG_mems.add_row([22,'',45.9313517,-15.4852220,'S0',None,None,2628,14.60,0.39,14.32,'NGC1188'])

HCG_mems.add_row([23,'',46.789543,-9.487844,'cI',None,None,5283,19.76,0.50,19.26,'HCG23-26'])

HCG_mems.add_row([31,'',75.433375,-4.288750,'cI',None,None,4011,15.49,0.5,15.11,'HCG31g'])
HCG_mems.add_row([31,'',75.409743,-4.222454,'cI',None,None,4090,16.23,0.50,16.01,'HCG31q'])

HCG_mems.add_row([49,'',164.160920,67.151693,'cI',None,None,9950,18.60,0.02,18.56,'HCG49SDSS1'])
#Velocity of HCG49SDSS1 is approximate (taken from HI cube)
#B-band luminosity estimated from SDSS bands via Lupton 2005 conversion

HCG_mems.add_row([54,'',172.36850671,20.6319153,'cI',None,None,1397,19.09,0.05,19.01,'A11272054'])

HCG_mems.add_row([71,'',212.769143,25.559671,'Sd',None,None,9083,18.21,0.5,17.97,'AGC732898'])
HCG_mems.add_row([71,'',212.725634,25.554015,'cI',None,None,9199,17.37,0.36,17.10,'AGC242021'])

HCG_mems.add_row([100,'',0.438449,13.100673,'cI',None,None,5606,15.77,0.05,15.34,'MRK935'])

In [ ]:
#Hickson used a different scheme for Hubble type
#Here we will use the RC3 system

T_type = []
for M_type in HCG_mems['MType']:
    if "E" in M_type:
        T_type.append(-5)
    elif "S0" in M_type or "SB0" in M_type:
        T_type.append(0)
    elif "Sa" in M_type or "SBa" in M_type or "SB-" in M_type:
        T_type.append(1)
    elif "Sab" in M_type or "SBab" in M_type:
        T_type.append(2)
    elif "Sb" in M_type or "SBb" in M_type:
        T_type.append(3)
    elif "Sbc" in M_type or "SBbc" in M_type:
        T_type.append(4)
    elif "Sc" in M_type or "SBc" in M_type:
        T_type.append(5)
    elif "Scd" in M_type or "SBcd" in M_type:
        T_type.append(6)
    elif "Sd" in M_type or "SBd" in M_type:
        T_type.append(7)
    elif "Sdm" in M_type or "SBdm" in M_type:
        T_type.append(8)
    elif "Sm" in M_type or "SBm" in M_type:
        T_type.append(9)
    elif "Im" in M_type:
        T_type.append(10)
    elif "cI" in M_type:
        T_type.append(11)
    else:
        print('Case not matched:')
        print(M_type)
        
HCG_mems['T'] = T_type

In [ ]:
HCG_mems = astropy.table.join(HCG_mems,HCGs['HCG','dist'],join_type='left')
HCG_mems.add_index('HCG')

In [ ]:
HCG_mems['logMHI_pred'],HCG_mems['e_logMHI_pred'] = numpy.round(logMHI_pred(numpy.array(HCG_mems['BTc']),numpy.array(HCG_mems['dist']),HCG_mems['e_BTmag'],0.),decimals=2)
HCG_mems['logMHI_pred_type'] = numpy.round(logMHI_pred_type(numpy.array(HCG_mems['BTc']),numpy.array(HCG_mems['dist']),numpy.array(HCG_mems['T'])),decimals=2)
HCG_mems['logMHI_pred_HG84'] = numpy.round(logMHI_pred_HG84(numpy.array(HCG_mems['BTc']),numpy.array(HCG_mems['dist'])),decimals=2)
HCG_mems['dist'] = numpy.round(HCG_mems['dist'],decimals=1)
HCG_mems['dist'].unit = units.Mpc 

In [ ]:
HCG_mems.pprint(max_lines=-1,max_width=-1)

In [ ]:
HI_pred = []
e_HI_pred = []
HI_pred_type = []
HI_pred_HG84 = []
for HCG in HCGs['HCG']:
    HI_pred.append(numpy.log10(numpy.sum(10.**HCG_mems.loc[HCG]['logMHI_pred'])))
    e_HI_pred.append(numpy.sqrt(numpy.sum(numpy.square(HCG_mems.loc[HCG]['e_logMHI_pred']*10.**HCG_mems.loc[HCG]['logMHI_pred'])))/numpy.sum((10.**HCG_mems.loc[HCG]['logMHI_pred'])))
    HI_pred_type.append(numpy.log10(numpy.sum(10.**HCG_mems.loc[HCG]['logMHI_pred_type'])))
    HI_pred_HG84.append(numpy.log10(numpy.sum(10.**HCG_mems.loc[HCG]['logMHI_pred_HG84'])))
HCGs['cz'] = numpy.array((numpy.round(HCGs['cz'],decimals=0)),dtype='int')
HCGs['cz'].unit = units.km/units.s 
HCGs['dist'] = numpy.round(HCGs['dist'],decimals=1)
HCGs['dist'].unit = units.Mpc 
HCGs['logMHI_pred'] = numpy.round(HI_pred,decimals=2)
HCGs['e_logMHI_pred'] = numpy.round(e_HI_pred,decimals=2)
HCGs['logMHI_pred_type'] = numpy.round(HI_pred_type,decimals=2)
HCGs['logMHI_pred_HG84'] = numpy.round(HI_pred_HG84,decimals=2)

In [ ]:
HCGs

In [ ]:
HCG_mems = HCG_mems['HCG','m_HCG','name','ra','dec','MType','T','a','b','Vel','BTmag','e_BTmag','BTc','dist','logMHI_pred','e_logMHI_pred']

In [ ]:
HCG_mems.write('tables/HCG_members.vo',format='votable',overwrite=True)
HCG_mems.write('tables/HCG_members.fits',format='fits',overwrite=True)
HCG_mems.write('tables/HCG_members.ascii',format='ascii.fixed_width_two_line',overwrite=True)

In [ ]:
HCGs.write('tables/HCGs.vo',format='votable',overwrite=True)
HCGs.write('tables/HCGs.fits',format='fits',overwrite=True)
HCGs.write('tables/HCGs.ascii',format='ascii.fixed_width_two_line',overwrite=True)